In [9]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

In [10]:
def euclidean_distance(tensor_a, tensor_b, dim = 0):
    return (tensor_b - tensor_a).pow(2).sum(dim).sqrt()

In [11]:
input = torch.randn(256, 128, 1, requires_grad=True)
target = torch.rand(256, 1, 1)
target = target.mul(9)
target = target.absolute()
target = target.round()

classes = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [12]:
input[0]

tensor([[ 0.1914],
        [ 1.2628],
        [ 0.0871],
        [ 0.9749],
        [ 0.0387],
        [ 0.8495],
        [-0.7429],
        [ 1.9241],
        [-0.0908],
        [-0.0222],
        [-0.1082],
        [-1.4186],
        [ 0.7260],
        [ 1.2859],
        [-0.0518],
        [ 1.3867],
        [ 0.4198],
        [-1.3410],
        [ 1.1393],
        [ 0.6238],
        [-0.0919],
        [ 1.8210],
        [-0.2215],
        [ 0.1737],
        [-1.2789],
        [-1.4505],
        [-2.5339],
        [ 0.0416],
        [ 1.0505],
        [ 0.3185],
        [-1.3754],
        [ 0.7929],
        [ 0.3113],
        [-0.9262],
        [-1.5360],
        [-0.2094],
        [ 1.5960],
        [ 0.0567],
        [ 0.1996],
        [-0.7823],
        [-0.1736],
        [ 0.6083],
        [ 0.4868],
        [ 0.2961],
        [-0.6717],
        [-0.5577],
        [-0.2424],
        [-0.8633],
        [-0.6058],
        [ 0.4986],
        [-0.4170],
        [-0.0305],
        [ 0.

In [17]:
def prepare(input, target, classes):
    batch_size = input.shape[0]
    # batch_size
    centroids = []
    for _class in classes:
        class_tensors = []
        for i in range(batch_size):
            if target[i] == _class:
                class_tensors.append(input[i])
        stacked_tensor = torch.stack(class_tensors)
        centroids.append(torch.mean(stacked_tensor, dim = 0))

    # self.centroids = torch.stack(centroids)

    # Jedes Element in centroids ist weiterhin ableitbar, aber centroids darf selbst nicht weiterverwendet werden.

    distances = []
    for centroid_a in centroids:
        local_distances = []
        for centroid_b in centroids:
            if not torch.equal(centroid_a, centroid_b):
                local_distances.append(euclidean_distance(centroid_a, centroid_b))
        local_tensor = torch.stack(local_distances)
        distances.append(torch.mean(local_tensor))      

    # (centroids[0] - centroids[1]).pow(2).sum(0).sqrt()
    return distances, centroids

In [18]:
dis, cen = prepare(input, target, classes)

In [20]:
# # sample_losses = torch.Tensor([0])
# for sample, label in zip(input, target):
#     # sample_losses.append(dists[int(label)])
#     try:
#         sample_losses = torch.cat((sample_losses, dists[int(label)]), 0)
#     except:
#         sample_losses = dists[int(label)]
# # loss_tensor = torch.stack(sample_losses)
# sample_losses

torch.stack(dis)

tensor([3.9760, 3.2155, 3.0312, 2.9166, 3.3240, 3.1716, 3.2446, 2.9944, 3.2826,
        3.8618], grad_fn=<StackBackward0>)

In [44]:
distances = []
for inp, tar in zip(input, target):
    label = int(tar)
    corresponding_centroid = cen[label].detach()
    distances.append(euclidean_distance(inp, corresponding_centroid))

# distances
torch.max(torch.stack(distances))

tensor(13.0781, grad_fn=<MaxBackward1>)